# Exercises XP: W2_D3

### Exercise 1: Duplicate Detection and Removal

In [1]:
import pandas as pd

In [2]:
# Step 1: Load the Titanic dataset
df = pd.read_csv("train.csv")  # Charger le fichier train.csv

In [3]:
# Step 2: Check number of rows before removing duplicates
rows_before = df.shape[0]  # Nombre de lignes avant suppression
print("Rows before removing duplicates:", rows_before)

Rows before removing duplicates: 891


In [4]:
# Step 3: Detect duplicates
duplicates = df.duplicated()  # Renvoie True pour les lignes dupliquées
print("Number of duplicate rows:", duplicates.sum())

Number of duplicate rows: 0


In [5]:
# Step 4: Remove duplicate rows
df = df.drop_duplicates()  # Supprimer les doublons

In [6]:
# Step 5: Verify the number of rows after removal
rows_after = df.shape[0]
print("Rows after removing duplicates:", rows_after)

Rows after removing duplicates: 891


In [7]:
# Step 6: Confirmation
print("Number of rows removed:", rows_before - rows_after)

Number of rows removed: 0


### Exercise 2: Handling Missing Values

In [9]:
pip install scikit-learn

   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ------------------------- -------------- 6.8/10.7 MB 36.0 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 32.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ------- -------------------------------- 7.9/41.0 MB 39.0 MB/s eta 0:00:01
   --------------- ------------------------ 16.0/41.0 MB 38.8 MB/s eta 0:00:01
   ----------------------- ---------------- 23.6/41.0 MB 37.6 MB/s eta 0:00:01
   ----------------------------- ---------- 30.4/41.0 MB 36.5 MB/s eta 0:00:01
   ------------------------------------- -- 38.3/41.0 MB 36.6 MB/s eta 0:00:01
   ---------------------------------------  40.9/41.0 MB 37.0 MB/s eta 0:00:01
   ---------------------------------------- 41.0/41.0 MB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [11]:
# Load Titanic dataset
df = pd.read_csv("train.csv")

In [12]:
# Step 1: Check which columns have missing values
print(df.isnull().sum())  # Shows number of missing values per column

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [13]:
# Step 2: Drop rows with missing values (example: drop rows where Age is missing)
df_dropped = df.dropna(subset=['Age'])  # Remove only rows with missing 'Age'

In [14]:
# Step 3: Fill missing values with a constant (example: fill missing Embarked with 'Unknown')
df_filled_constant = df.copy()
df_filled_constant['Embarked'] = df_filled_constant['Embarked'].fillna('Unknown')

In [15]:
# Step 4: Impute missing numerical values with mean using SimpleImputer (example: Age)
imputer = SimpleImputer(strategy='mean')  # You can also try 'median' or 'most_frequent'
df_imputed = df.copy()
df_imputed[['Age']] = imputer.fit_transform(df_imputed[['Age']])

In [16]:
# Step 5: Show how many missing values remain (should be 0 for Age and Embarked)
print(df_imputed.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


### Exercise 3: Feature Engineering

In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [30]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

In [31]:
# Create a new feature 'FamilySize' by adding siblings/spouses (SibSp) and parents/children (Parch), plus 1 for self
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

In [32]:
# Extract the passenger's title (Mr, Mrs, Miss, etc.) from the 'Name' column
# The double backslash escapes the dot correctly
df["Title"] = df["Name"].str.extract(r'([A-Za-z]+)\.', expand=False)

In [33]:
# Group less frequent titles under a common label 'Rare'
rare_titles = df["Title"].value_counts()[df["Title"].value_counts() < 10].index
df["Title"] = df["Title"].replace(rare_titles, "Rare")

In [34]:
# Apply one-hot encoding to convert categorical variables into numeric format
# 'drop_first=True' removes one category to avoid multicollinearity
df = pd.get_dummies(df, columns=["Sex", "Embarked", "Title"], drop_first=True)

In [35]:
# Define the numerical columns to normalize
num_cols = ["Age", "Fare", "FamilySize"]

In [36]:
# Normalize the numerical columns using StandardScaler
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [37]:
# Show the first few rows of the transformed dataset
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name       Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris -0.530377      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  0.571831      1      0   
2                             Heikkinen, Miss. Laina -0.254825      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  0.365167      1      0   
4                           Allen, Mr. William Henry  0.365167      0      0   

             Ticket      Fare Cabin  FamilySize  Sex_male  Embarked_Q  \
0         A/5 21171 -0.502445   NaN    0.059160      True       False   
1          PC 17599  0.786845   C85    0.059160     False       False   
2  STON/O2. 3101282 -0.488854   NaN   -0.560975     False       False   
3         

###  Exercise 4: Outlier Detection and Handling

#### 4.1 Outlier Detection and Removal using Interquartile Range (IQR)

In [38]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [39]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

In [40]:
# Define a function to detect and remove outliers using the IQR method
def remove_outliers_iqr(data, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter the data to remove outliers
    filtered_data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    
    return filtered_data

In [41]:
# Store original size
original_shape = df.shape

In [42]:
# Remove outliers from 'Fare'
df = remove_outliers_iqr(df, 'Fare')

In [43]:
# Remove outliers from 'Age'
df = remove_outliers_iqr(df, 'Age')

In [44]:
# Print the shape before and after outlier removal
print("Original shape:", original_shape)
print("Shape after outlier removal:", df.shape)

Original shape: (891, 12)
Shape after outlier removal: (601, 12)


In [45]:
# Optional: show the summary statistics to see the effect
print(df[['Age', 'Fare']].describe())

              Age        Fare
count  601.000000  601.000000
mean    28.152662   18.610960
std     13.336564   13.669143
min      0.420000    0.000000
25%     20.000000    7.925000
50%     28.000000   13.000000
75%     36.000000   26.250000
max     62.000000   65.000000


#### 4.2 Outlier Detection and Removal using Z-score

In [46]:
import pandas as pd
from scipy import stats

In [47]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

In [53]:
# Drop rows with missing values in Age and Fare to avoid NaNs during Z-score calculation
df = df.dropna(subset=["Age", "Fare"])

In [54]:
# Compute Z-scores for Age and Fare
z_scores = stats.zscore(df[["Age", "Fare"]])

In [55]:
# Create a DataFrame of z-scores, using the same index as the original df to ensure alignment
z_df = pd.DataFrame(z_scores, columns=["Age_z", "Fare_z"], index=df.index)

In [56]:
# Keep only rows where both Age_z and Fare_z are between -3 and 3 (i.e., not outliers)
df_clean = df[(z_df["Age_z"].abs() < 3) & (z_df["Fare_z"].abs() < 3)]

In [57]:
# Display the number of rows before and after outlier removal
print(f"Rows before cleaning: {df.shape[0]}")
print(f"Rows after cleaning: {df_clean.shape[0]}")

Rows before cleaning: 714
Rows after cleaning: 694


In [58]:
# Optionally preview the cleaned dataset
print(df_clean.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


### Exercise 5: Data Standardization and Normalization

In [59]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [60]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

In [61]:
# Select the numerical columns we want to transform
# You may adjust these columns depending on your analysis
num_cols = ["Age", "Fare"]

In [62]:
# Drop rows with missing values in selected columns
df = df.dropna(subset=num_cols)


##### 1 Standardization using StandardScaler (mean = 0, std = 1)
##### This is useful for algorithms that assume data is normally distributed
##### ---------------------------------------------

In [63]:
standard_scaler = StandardScaler()
df_standardized = df.copy()  # Create a copy to store standardized data
df_standardized[num_cols] = standard_scaler.fit_transform(df_standardized[num_cols])

##### 1 Normalization using MinMaxScaler (values scaled between 0 and 1)
##### This is useful when you want to compare features with different units or scales
##### ---------------------------------------------

In [64]:
minmax_scaler = MinMaxScaler()
df_normalized = df.copy()  # Create another copy for normalized data
df_normalized[num_cols] = minmax_scaler.fit_transform(df_normalized[num_cols])

##### Show a preview of both versions for comparison

In [68]:
print("Standardized Data:")
print(df_standardized[num_cols].head())

Standardized Data:
        Age      Fare
0 -0.530377 -0.518978
1  0.571831  0.691897
2 -0.254825 -0.506214
3  0.365167  0.348049
4  0.365167 -0.503850


In [69]:
print("\nNormalized Data:")
print(df_normalized[num_cols].head())


Normalized Data:
        Age      Fare
0  0.271174  0.014151
1  0.472229  0.139136
2  0.321438  0.015469
3  0.434531  0.103644
4  0.434531  0.015713


### Exercise 6: Feature Encoding

In [70]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [71]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

##### ---------------------------------------------
##### 1 Identify categorical columns
##### We'll focus on 'Sex' and 'Embarked' for nominal variables
##### And simulate 'Pclass' as an ordinal variable (already encoded as 1, 2, 3)
##### ---------------------------------------------

In [72]:
# Drop rows with missing values in 'Embarked' (to avoid issues when encoding)
df = df.dropna(subset=["Embarked"])

##### ---------------------------------------------
##### 2 One-Hot Encoding for Nominal Variables
##### This will create new binary columns for each category
##### ---------------------------------------------

In [73]:
df_encoded = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)

##### ---------------------------------------------
##### 3 Label Encoding for Ordinal Variable (simulated here with 'Pclass')
##### Normally, you'd use this if values have an order like: Low < Medium < High
##### But 'Pclass' is already numeric, so we show the usage with a new example
##### ---------------------------------------------

In [ ]:
# Example: suppose we have a custom ordinal feature
# df["CabinClass"] = ["Basic", "Premium", "Luxury", "Basic", "Luxury"]
# label_encoder = LabelEncoder()
# df["CabinClass_encoded"] = label_encoder.fit_transform(df["CabinClass"])

In [74]:
df = pd.DataFrame({
    "CabinClass": ["Basic", "Premium", "Luxury", "Basic", "Luxury"]
})

label_encoder = LabelEncoder()
df["CabinClass_encoded"] = label_encoder.fit_transform(df["CabinClass"])

print(df)

  CabinClass  CabinClass_encoded
0      Basic                   0
1    Premium                   2
2     Luxury                   1
3      Basic                   0
4     Luxury                   1


In [75]:
# Display the encoded dataset
print(df_encoded[["Sex_male", "Embarked_Q", "Embarked_S"]].head())

   Sex_male  Embarked_Q  Embarked_S
0      True       False        True
1     False       False       False
2     False       False        True
3     False       False        True
4      True       False        True


### Exercise 7: Data Transformation for Age Feature

In [76]:
import pandas as pd

In [77]:
# Load the Titanic dataset
df = pd.read_csv("train.csv")

In [78]:
# Drop rows with missing Age values (to avoid issues during binning)
df = df.dropna(subset=["Age"])

In [79]:
# 1. Create age groups using pd.cut()
# This will divide the Age column into 5 categories: Child, Teenager, Young Adult, Adult, and Senior
age_bins = [0, 12, 18, 35, 60, 100]
age_labels = ["Child", "Teenager", "Young Adult", "Adult", "Senior"]
df["AgeGroup"] = pd.cut(df["Age"], bins=age_bins, labels=age_labels)

In [80]:
# 2. Apply one-hot encoding to the AgeGroup column
df = pd.get_dummies(df, columns=["AgeGroup"], prefix="AgeGroup")

In [81]:
# 3. Preview the result
print(df[["Age", "AgeGroup_Child", "AgeGroup_Teenager", "AgeGroup_Young Adult", "AgeGroup_Adult", "AgeGroup_Senior"]].head())

    Age  AgeGroup_Child  AgeGroup_Teenager  AgeGroup_Young Adult  \
0  22.0           False              False                  True   
1  38.0           False              False                 False   
2  26.0           False              False                  True   
3  35.0           False              False                  True   
4  35.0           False              False                  True   

   AgeGroup_Adult  AgeGroup_Senior  
0           False            False  
1            True            False  
2           False            False  
3           False            False  
4           False            False  


#### Explanation
##### pd.cut() is used to divide continuous values (like Age) into intervals or "bins".
##### labels=age_labels assigns human-readable names to each age bin.
##### pd.get_dummies() converts each age category into a binary feature column (e.g., AgeGroup_Child = 1 if the person is a child).
##### We dropped rows with missing Age to avoid errors during binning.